In [23]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [22]:
# dogs: 22 - 26
# cats: 27 - 29
dataset = load_dataset("zh-plus/tiny-imagenet")

In [58]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("zh-plus/tiny-imagenet")['train']
        else:
            temp_dataset = load_dataset("zh-plus/tiny-imagenet")['valid']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] in [27, 28, 29]:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['image']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.randint(0, 1900, (1, )) < 10:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['image']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = torchvision.transforms.RandomHorizontalFlip()(image)
        return (image, label)

In [39]:
train_dataset = CatsDataset()

In [40]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

In [50]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [51]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 5),
    nn.MaxPool2d((5, 5)),
    nn.LeakyReLU(),
    nn.Conv2d(16, 64, 5),
    nn.MaxPool2d((5, 5)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(64, 2)
).to(dev)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [52]:
for epoch in range(40):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.6031195027753711
Epoch 2 Loss: 0.5693937763571739
Epoch 3 Loss: 0.5645777573809028
Epoch 4 Loss: 0.5563384471461177
Epoch 5 Loss: 0.5256901932880282
Epoch 6 Loss: 0.519495652988553
Epoch 7 Loss: 0.5277099274098873
Epoch 8 Loss: 0.4949968680739403
Epoch 9 Loss: 0.4846584042534232
Epoch 10 Loss: 0.4662678251042962
Epoch 11 Loss: 0.46639192570000887
Epoch 12 Loss: 0.4956228667870164
Epoch 13 Loss: 0.45122461300343275
Epoch 14 Loss: 0.43881895393133163
Epoch 15 Loss: 0.4320981102064252
Epoch 16 Loss: 0.43389446567744017
Epoch 17 Loss: 0.4174723094329238
Epoch 18 Loss: 0.4161420054733753
Epoch 19 Loss: 0.4145952542312443
Epoch 20 Loss: 0.4002936128526926
Epoch 21 Loss: 0.3905644016340375
Epoch 22 Loss: 0.40552608389407396
Epoch 23 Loss: 0.38886744529008865
Epoch 24 Loss: 0.38923065550625324
Epoch 25 Loss: 0.3654237175360322
Epoch 26 Loss: 0.38696625735610723
Epoch 27 Loss: 0.3586164745502174
Epoch 28 Loss: 0.34843491297215223
Epoch 29 Loss: 0.3499384238384664
Epoch 30 Loss: 

In [59]:
test_dataset = CatsDataset(train=False)

In [60]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [61]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.8029, device='mps:0')
